In [7]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
!pip install folium shapely pandas

StatementMeta(, 86bfecc3-e60d-40ab-b5b6-881cca06649e, 9, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.2 MB/s eta 0:00:00a 0:00:01


In [10]:

import pandas as pd
import folium
from shapely import wkt
from shapely.geometry import mapping
import json
import branca.colormap as bcm

# Load the CSV
url =  "https://raw.githubusercontent.com/riyaziqbal/toronto-equity-dashboard/refs/heads/main/data/processed/silver_03_dim_ward_demo_geo_toronto.csv"

df = pd.read_csv(url)
print(df.dtypes)

# Parse WKT geometry
df["geometry_wkt"] = df["geometry_wkt_simplified"].apply(wkt.loads)

# Create base map
m = folium.Map(location=[43.7, -79.4], zoom_start=10, control_scale=True, tiles=None)
# Adding more tile layers to the base map
folium.TileLayer("cartodbpositron", name="Toronto Base Map").add_to(m)
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
folium.TileLayer("Stamen Toner", name="Toner").add_to(m)
folium.TileLayer("Esri.WorldTopoMap", name="Terrain (Esri)").add_to(m)

# Create separate layers
income_layer = folium.FeatureGroup(name="Average Income")
median_layer = folium.FeatureGroup(name="Median Income")
population_layer = folium.FeatureGroup(name="Population")

# Create choropleths based on income
# # Normalize income for color scaling

min_income = df["average_income"].min()
max_income = df["average_income"].max()

# Create a smooth sequential colormap
income_colormap = bcm.linear.PuBuGn_09.scale(min_income, max_income)  # Purple Blue Green
# colormap = bcm.linear.Blues_09.scale(min_income, max_income)  # Blues
# colormap = bcm.linear.YlGn_09.scale(min_income, max_income)  # Yellow Green
income_colormap.caption = "Average Income per Ward"

# Add ward polygons with choropleth coloring to Average Income layer
for _, row in df.iterrows():
    geo_json = mapping(row["geometry_wkt"])
    fill_color = income_colormap(row["average_income"])
    tooltip = f"Ward: {row['ward_name']}<br>Income: ${row['average_income']:,.0f}"
    
    folium.GeoJson(
        geo_json,
        tooltip=tooltip,
        style_function=lambda x, color=fill_color: {
            "fillColor": color,
            "color": "white",
            "weight": 1,
            "fillOpacity":0.8
        }
    ).add_to(income_layer)

#Normalize population for color scaling

max_pop =df["population"].max()
min_pop =df["population"].min()

# Create a smooth sequential colormap
population_colormap = bcm.linear.PuBuGn_09.scale(min_pop, max_pop)  # Purple Blue Green
population_colormap.caption = "Population per Ward"

for _, row in df.iterrows():
    geo_json = mapping(row["geometry_wkt"])
    color = population_colormap(row["population"])
    tooltip = f"Ward: {row['ward_name']}<br>Population: ${row['population']:,.0f}"
    
    folium.GeoJson(
        geo_json,
        tooltip=tooltip,
        style_function=lambda x, color=color: {
            "fillColor": color,
            "color": "white",
            "weight": 1,
            "fillOpacity": 0.8
        }
    ).add_to(population_layer)


#Normalize median income for color scaling

max_median_income =df["median_income"].max()
min_median_income =df["median_income"].min()

# Create a smooth sequential colormap
median_colormap= bcm.linear.PuBuGn_09.scale(min_median_income, max_median_income)  # Purple Blue Green
median_colormap.caption = "Median Income per Ward"

for _, row in df.iterrows():
    geo_json = mapping(row["geometry_wkt"])
    color = median_colormap(row["median_income"])
    tooltip = f"Ward: {row['ward_name']}<br>Median Income: ${row['median_income']:,.0f}"
    
    folium.GeoJson(
        geo_json,
        tooltip=tooltip,
        style_function=lambda x, color=color: {
            "fillColor": color,
            "color": "white",
            "weight": 1,
            "fillOpacity": 0.8
        }
    ).add_to(median_layer)

income_layer.add_to(m)
median_layer.add_to(m)
population_layer.add_to(m)

# Add legends
income_colormap.add_to(m)
population_colormap.add_to(m)
median_colormap.add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

m


StatementMeta(, 86bfecc3-e60d-40ab-b5b6-881cca06649e, 12, Finished, Available, Finished)

ward_id                      int64
ward_name                   object
average_income             float64
median_income              float64
population                   int64
geometry_wkt_simplified     object
dtype: object


## **Add TTC Stops Data**

In [42]:
import pandas as pd

# Reading TTC stops data
ttc_stops = "https://raw.githubusercontent.com/riyaziqbal/toronto-equity-dashboard/refs/heads/main/data/raw/stops.txt"
df_ttc = pd.read_csv(ttc_stops)
df_ttc.head(1)

df_sample = df_ttc.sample(n=2000, random_state=42)  # or filter by route/ward

from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster(name="TTC Stops").add_to(m)

for _, stop in df_sample.iterrows():
    coords = (stop["stop_lat"], stop["stop_lon"])
    folium.Marker(coords, tooltip=stop["stop_name"]).add_to(marker_cluster)

m.save("ttc_map.html")

StatementMeta(, 86bfecc3-e60d-40ab-b5b6-881cca06649e, 44, Finished, Available, Finished)

com.fasterxml.jackson.core.exc.StreamConstraintsException: String length (20054016) exceeds the maximum length (20000000)